In [1]:
import numpy as np

In [25]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
Z = 'red'
pHit = 0.6
pMiss = 0.2

In [13]:
def sense(p, Z):
    prob = np.array(p)
    measure = np.array([pHit if i == Z else pMiss for i in world])
    
    combine_prob = prob*measure
    
    norm_prob = combine_prob/sum(combine_prob)
    return norm_prob

In [11]:
sense(p, Z)

array([0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111])

In [14]:
Z = 'green'

sense(p, Z)

array([0.27272727, 0.09090909, 0.09090909, 0.27272727, 0.27272727])

In [26]:
measurements = ['red', 'green']

for i in measurements:
    p = sense(p, i)
    print("sensing [{}] -> prior distribution {}".format(i, p))

sensing [red] -> prior distribution [0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
sensing [green] -> prior distribution [0.2 0.2 0.2 0.2 0.2]


In [34]:
def move(p, U):
    """
    p: probability distribution 
    U: number of steps moving
    """
    n = len(p)
    U = U % n
    
    return p[-U:] + p[:(n-U)]

In [37]:
p = [0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111]

move(p, -1)

[0.33333333, 0.33333333, 0.11111111, 0.11111111, 0.11111111]

### Non-accurate Move

In [39]:
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [44]:
def move(p, U):
    """
    movement inaccurate
    p: probability distribution 
    U: number of steps moving
    """
    p = np.array(p)
    move_prob = np.array([pOvershoot, pExact, pUndershoot])
    n = len(p)
    U = U % n
    
    q = []
    for i in range(n):
        steps = [i-U-1, i-U, i-U+1]
        q_prob = np.dot(p[steps], move_prob)
        q.append(q_prob)
    return q

In [46]:
p = [0, 1, 0, 0, 0]
move(p, 1)

[0.0, 0.1, 0.8, 0.1, 0.0]